# 4장. 좋은 훈련 데이터셋 만들기: 데이터 전처리

* 데이터셋에서 누락된 값을 제거하거나 대체하기
* 머신 러닝 알고리즘을 위해 범주형 데이터 변환하기
* 모델과 관련이 높은 특성 선택하기

## 4.1 누락된 데이터 다루기

데이터셋에서 샘플을 제거하거나 다른 샘플이나 특성에서 누락된 값을 대체하는 기법

### 4.1.1 테이블 형태 데이터에서 누락된 값 식별
